# Machinaal Leren - Sprint 2: Text Data

## Task:
What insights can we gain from the text data (title, description and reviews) ?

Possible tasks:
* Detect duplicate listings
* Extract keywords from reviews / descriptions
* Automatically make a list of positive and negative points for a listing based on the reviews
* Recognize listings from the same owner
* Detect anomalies (listings/ reviews that are very different from other listings/ reviews)
* Detect reviews that are very similar
* Perform sentiment analysis on a review
* ...

## Work table


|Task|Peter Bonnarens|Philip Kukoba|Lennert Franssens|
|------|------|------|------|
|Detect duplicate listings  |X  |X  | X |
|Extract keywords  |_  |_  | _ |
|Automatically make a list  |_  |_  | _ |
|Recognize listings from the same owner  |_  |_  | _ |
|Detect anomalies  |_  |_  | _ |
|Detect reviews that are very similar  |_  |_  | _ |
|Perform sentiment analysis on a review  |_  |_  | _ |

# 1 - Loading the dataset

In [ ]:
%matplotlib inline

# imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd  
import seaborn as sns 
from matplotlib import rcParams
import warnings

# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', -1)
# delete warnings from output
warnings.filterwarnings('ignore')

# figure size in inches
plt.rcParams['figure.figsize'] = 15, 12

# loading the datasets into pandas dataframes
reviews = pd.read_csv("data/reviews.csv")
listings = pd.read_csv("data/listings.csv")

In [ ]:
reviews.head()

In [ ]:
listings.head()

Find lines to shift and add them to a mask - we've found that some lines are shifted 1 to the right beginning on the host_id column (that now contains garbage data)

In [ ]:
shifted_lines = listings[pd.to_numeric(listings["host_verifications"], errors='coerce').notnull()].id
mask = listings['id'].isin(shifted_lines)

# shift lines 1 to the left
listings.loc[mask, 'host_since':'reviews_per_month'] = listings.loc[mask, 'host_since':'reviews_per_month'].shift(-1, axis=1)

In [ ]:
listings.drop(['host_response_time','host_response_rate','host_acceptance_rate','host_is_superhost','host_listings_count','host_total_listings_count',
            'host_has_profile_pic','host_identity_verified','latitude','longitude','accommodates','bathrooms','bedrooms','beds','price','minimum_nights',
            'maximum_nights','minimum_minimum_nights','maximum_minimum_nights','minimum_maximum_nights','maximum_maximum_nights','minimum_nights_avg_ntm',
            'maximum_nights_avg_ntm','calendar_updated','has_availability','availability_30','availability_60','availability_90','availability_365',
            'number_of_reviews','number_of_reviews_ltm','number_of_reviews_l30d','first_review','last_review','license','instant_bookable','calculated_host_listings_count',
            'calculated_host_listings_count_entire_homes','calculated_host_listings_count_private_rooms','calculated_host_listings_count_shared_rooms',
            'reviews_per_month'], axis=1)

Drop all rows without comments.

In [ ]:
reviews = reviews.dropna()
reviews_points = reviews.copy()

# Remove <b> tags and text in between it
reviews['comments'] = reviews['comments'].str.replace(r'<b>.*?<\/b>', '', regex=True)
reviews_points['comments'] = reviews_points['comments'].str.replace(r'<b>.*?<\/b>', '', regex=True)

# Remove all HTML tags
reviews['comments'] = reviews['comments'].str.replace(r'<[^<>]*>', ' ', regex=True)
reviews_points['comments'] = reviews_points['comments'].str.replace(r'<[^<>]*>', ' ', regex=True)

# Remove all special characters (non-word)
reviews['comments'] = reviews['comments'].str.replace(r'\W', ' ', regex=True)
reviews_points['comments'] = reviews_points['comments'].str.replace(r'[^\w\s\'\.\/]', '.', regex=True)

# Remove all single characters (like a floating 'b' after removing the HTML tags around it)
reviews['comments'] = reviews['comments'].str.replace(r'\s+[a-zA-Z]\s+', ' ', regex=True)
reviews_points['comments'] = reviews_points['comments'].str.replace(r'\s+[a-zA-Z]\s+', ' ', regex=True)

# Remove single characters at the start
reviews['comments'] = reviews['comments'].str.replace(r'\^[a-zA-Z]\s+', ' ', regex=True)
#reviews_points['comments'] = reviews_points['comments'].str.replace(r'\^[a-zA-Z]\s+', ' ', regex=True)

# Substitute multiple spaces with a single space
reviews['comments'] = reviews['comments'].str.replace(r'\s+', ' ', regex=True)
reviews_points['comments'] = reviews_points['comments'].str.replace(r'\s+', ' ', regex=True)

# Replace multiple dots with one dot
reviews_points['comments'] = reviews_points['comments'].str.replace(r'\.+', '.', regex=True)

# Remove space(s) before dot
reviews_points['comments'] = reviews_points['comments'].str.replace(r'\s+\.', '.', regex=True)

# Convert to lowercase
reviews['comments'] = reviews['comments'].str.lower()
reviews_points['comments'] = reviews_points['comments'].str.lower()

In [ ]:
# Remove <b> tags and text in between it
listings['description'] = listings['description'].str.replace(r'<b>.*?<\/b>', '', regex=True)

# Remove all HTML tags
listings['description'] = listings['description'].str.replace(r'<[^<>]*>', ' ', regex=True)
listings = listings[listings['description'].notna()]

## Detect duplicate listings
To detect duplicates we compare entries based on the description column. The column should be carefully chosen since e.g. comparing based on the name might mark non-duplicates as duplicates, if both have the same generic name.

In [ ]:
print(listings[listings.duplicated(['description']) == True])

## Positive and negative points from a listing review

### Import packages

In [ ]:
# run this in your environment: python -m pip install textblob
import nltk
#nltk.download('punkt')
from textblob import TextBlob

### Apply TextBlob

In [ ]:
reviews_points['comments'] = reviews_points['comments'].apply(TextBlob)

In [ ]:
for index, row in reviews_points.iterrows():
    comment = row.comments
    positive = []
    negative = []
    for sentence in comment.sentences:
        if sentence.sentiment.polarity < -0.3:
            negative.append(str(sentence).replace('.', ''))
        elif sentence.sentiment.polarity > 0.3:
            positive.append(str(sentence).replace('.', ''))
    print("\n----- COMMENT -----")
    print("listing_id: ", row.listing_id)
    print("comment: ", comment)
    print("positive points: ", positive)
    print("negative points: ", negative)

## Extract keywords from reviews using TF-IDF

TODO WRITE INTRO

TODO EXTEND FOR DESCRIPTION


Before running TF-IDF, we first have to detect the language of each review (since for example stop words are unique for a language). Be warned, the code below takes a while to compute.

In [249]:
# run this in your environment: python -m pip install langdetect

from langdetect import detect

all_languages = []
for review in reviews["comments"].values:
    try:
        all_languages.append(detect(review))
    except: #the detect func rarely fails, but when it does it can crash the for loop
        all_languages.append("unknown")
# all_languages

We add a column to our dataframe and look at the results.

In [ ]:
reviews["language"] = all_languages
reviews["language"].value_counts().head()

Most reviews are in English (en). We will keep only the English reviews since most libraries are optimal for English (eg stop words for tf-idf).

In [ ]:
english_reviews = reviews[reviews["language"] == 'en']
english_reviews

We set-up the CountVectorizer and TfidTransformer properly for TF-IDF.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

count_vect = CountVectorizer(
    stop_words="english",
    ngram_range=(1,2),
    min_df=2,    # todo experiment with different values
    max_df=0.5
)

X_train_counts = count_vect.fit_transform(english_reviews["comments"])

tf_transformer = TfidfTransformer()
X_train_tf = tf_transformer.fit_transform(X_train_counts) # is a matrix

X_train_tf   # matrix of #entries x #words(features)

We will add a column to our DataFrame with the top 5 keywords for this comment based on TF-IDF.

In [ ]:
feature_names=count_vect.get_feature_names()


#todo fix this... at the end it uses a too large index for some reason

top_5_keywords_column = []
i = 0 
for review in reviews["comments"]: 
    print(i)
    np_arr = np.array(X_train_tf[i])
    top_5_tf_idf_indices = np.array(X_train_tf[i]).argsort()[:5]
    #print("got here")
    
    top_5_keywords = []
    for ind in top_5_tf_idf_indices:
        top_5_keywords.append(feature_names[ind])
    
    top_5_keywords_column.append(top_5_keywords)
    
    i += 1
    
reviews_english["top_5_keywords"] = top_5_keywords_column